In [4]:
import re
from datetime import datetime
import PyPDF2
import pandas as pd
import os
import email
import imaplib
import locale
from locale import atof
import json

# Configuración del locale
locale.setlocale(locale.LC_NUMERIC, 'es_ES.UTF-8')

json_file_path= "C:/Users/esteb/Documents/Automatizacion/MutualFund_Info.json"
with open(json_file_path, 'r') as file:
    # Carga los datos JSON del archivo
    key_info = json.load(file)

# Connect to the Gmail server
mail = imaplib.IMAP4_SSL('imap.gmail.com')
mail.login(key_info["email"], key_info["email_code"])
mail.select('inbox')

# Search for emails from a specific sender received since the first day of two months ago
tipo, data = mail.search(None, f'(FROM "{key_info["Nu_email"]}")')

nu_path = key_info["Path_NuBank"]

# Make sure the directories exist
os.makedirs(nu_path, exist_ok=True)

# Procesar correos y descargar los archivos
for num in data[0].split():
    typ, msg_data = mail.fetch(num, '(RFC822)')
    for response_part in msg_data:
        if isinstance(response_part, tuple):
            msg = email.message_from_bytes(response_part[1])
            for part in msg.walk():
                if part.get_content_maintype() == 'multipart':
                    continue

                # Check if part has Content-Disposition header (indicative of an attachment)
                if part.get('Content-Disposition') is not None:
                    filename = part.get_filename()
                    if filename:
                        print(filename)
                        # Determine the correct path based on the filename
                        if any(keyword in filename.upper() for keyword in ["NU"]):
                            filepath = os.path.join(nu_path, filename)

                        # Check if file already exists
                        if not os.path.isfile(filepath):
                            with open(filepath, 'wb') as fp:
                                fp.write(part.get_payload(decode=True))
                            print(f'Saved {filename}')
                        else:
                            print(f'File {filename} already exists. Skipping download.')

# Close the mail connection
mail.close()
mail.logout()

Tu Contrato Nu.pdf
Saved Tu Contrato Nu.pdf
Nu_2022-10-05.pdf
Saved Nu_2022-10-05.pdf
Nu_2022-11-05.pdf
Saved Nu_2022-11-05.pdf
Nu_2023-02-05.pdf
Saved Nu_2023-02-05.pdf
Nu_2023-03-05.pdf
Saved Nu_2023-03-05.pdf
Nu_2023-04-05.pdf
Saved Nu_2023-04-05.pdf
Nu_2023-05-05.pdf
Saved Nu_2023-05-05.pdf
Nu_2023-06-05.pdf
Saved Nu_2023-06-05.pdf
Nu_2023-07-05.pdf
Saved Nu_2023-07-05.pdf
Nu_2023-08-05.pdf
Saved Nu_2023-08-05.pdf
Nu_2023-09-05.pdf
Saved Nu_2023-09-05.pdf
Nu_2023-10-05.pdf
Saved Nu_2023-10-05.pdf
certificado_retencion_nu_2022_2023-09-25.pdf
Saved certificado_retencion_nu_2022_2023-09-25.pdf
Nu_2023-11-23.pdf
Saved Nu_2023-11-23.pdf
Nu_2023-12-23.pdf
Saved Nu_2023-12-23.pdf
Nu_2024-01-23.pdf
Saved Nu_2024-01-23.pdf
Nu_2024-02-23.pdf
Saved Nu_2024-02-23.pdf
Nu_2024-03-23.pdf
Saved Nu_2024-03-23.pdf
Nu_2024-04-23.pdf
Saved Nu_2024-04-23.pdf
Nu_2024-05-23.pdf
Saved Nu_2024-05-23.pdf
Nu_2024-06-23.pdf
Saved Nu_2024-06-23.pdf
Nu_2024-07-23.pdf
Saved Nu_2024-07-23.pdf
Nu_2024-08-23.pdf
Sa

('BYE', [b'LOGOUT Requested'])

In [11]:
carpeta = key_info["Path_NuBank"]
password = key_info["Id"]

# Lista para almacenar el texto de todos los archivos PDF
textos = []

# Iterar a través de cada archivo en la carpeta
for archivo in os.listdir(carpeta):
    ruta_archivo = os.path.join(carpeta, archivo)
    # Asegurarse de que el archivo sea un PDF
    if ruta_archivo.lower().endswith('.pdf'):
        # Abrir el archivo PDF
        with open(ruta_archivo, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            # Verificar si el archivo PDF está encriptado
            if reader.is_encrypted:
                # Intentar desencriptar el PDF
                if reader.decrypt(password):
                    # Inicializar una variable para almacenar el texto de todas las páginas
                    texto_completo = ''
                    # Extraer texto de cada página y concatenarlo
                    for page in reader.pages:
                        text = page.extract_text()
                        if text:
                            texto_completo += text + '\n'  # Añadir un salto de línea entre páginas
                    # Añadir el texto completo del PDF a la lista 'textos'
                    textos.append(texto_completo)
                else:
                    print(f"Contraseña incorrecta o imposible desencriptar el PDF: {archivo}")
            else:
                for page in reader.pages:
                    text = page.extract_text()
                    if text:
                        texto_completo += text + '\n'  # Añadir un salto de línea entre páginas
                    # Añadir el texto completo del PDF a la lista 'textos'
                textos.append(texto_completo)

# Supongamos que esta es tu lista de textos (cada elemento es el contenido de un archivo PDF)
lista_textos = textos

In [12]:
len(lista_textos)

23